# Capstone Data Project

## Yong (Daniel) Shen

### This notebook will be primarily used for the Coursera IBM Data Science Capstone Project

In [92]:
import pandas as pd
import numpy as np

In [93]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 1. Segmenting and Clustering Toronto Data

In [101]:
#Here, I use the BeautifulSoup package to scrape the Wikipedia page data and use Pandas to clean it.

import requests
from bs4 import BeautifulSoup

wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wiki.content,'lxml')
table = soup.find_all('tr')
list_rows = []
for row in table:
    row_td = row.find_all('td')
    str_cells = str(row_td)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    
    list_rows.append(cleantext)
#print(row_td)
#print(list_rows)

In [102]:
#Building Pandas dataframe to store the scraped raw data into manageable format
df = pd.DataFrame(list_rows)
#df
#After checking df, I relized that there are extra rows that are taken that aren't needed and should be cleaned. So I make another dataframe.
df1 = df[1:288]
df1.head()

,0
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"


In [103]:
#Split the three into columns, get rid of the brackets, and get rid of the \n in the end of each row
df1 = df1[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[]')
df1[2] = df1[2].str.strip('\n]')
#We are only interested in the first 3 columns so we'll take the first three with the following code
df1 = df1.iloc[:, 0:3]
df1.columns = ['PostalCode', 'Borough', 'Neighborhood']
df1['Borough'].str.strip()
#df1.head()
df1.shape

(287, 3)

In [176]:
#Now we can get rid of the Boroughs that have value = "not assigned"
i = df1['Borough'] == " Not assigned"
no_borough_names = df1[i].index
df1.drop(no_borough_names, inplace=True)
df1.reset_index(drop=True, inplace=True)
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [177]:
#Now let's assign the borough name to the neighborhood name if the neighborhood name is unassigned.
a = df1['Neighborhood'] == " Not assigned"
no_neigh_names = df1[a].index
df1.iloc[no_neigh_names, 2] = df1.iloc[no_neigh_names, 1]

In [178]:
# Now we combine the Neighborhoods with the same PostalCode into one row, separated by commas.
df2 = df1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list)
df3 = pd.DataFrame(df2)
df3.reset_index(inplace=True)
df3['Neighborhood'] = df3['Neighborhood'].apply(lambda x: ','.join(map(str, x)))
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [179]:
#Now we print the shape of the cleaned dataframe after applying the .shape function
df3.shape

(103, 3)